# Comparing various values from ISIS and USGSCSM cameras for a Messenger MDIS NAC PDS3 image

In [1]:
import os
os.environ['ISISROOT'] = '/usr/local/Caskroom/miniconda/base/envs/isis-prod'
os.environ['ISISDATA'] = '/Volumes/usgs-data/isis-data'
import urllib.request

import ale
from ale.drivers.mess_drivers import MessengerMdisPds3NaifSpiceDriver
from ale.formatters.usgscsm_formatter import to_usgscsm
import ale.isd_generate
import json
import os
import kalasiris as isis
import pvl
import numpy as np
import knoten
import csmapi
from knoten import csm

ale.spice_root = '/Volumes/usgs-data/spice-data'

In [2]:
# https://pdsimage2.wr.usgs.gov/Messenger/MSGRMDS_1001/DATA/2015_114/
# https://d3fhgbbgskqro0.cloudfront.net/MSGRMDS_1001/DATA/2015_114/EN1072174528M.IMG

imgurl = 'https://d3fhgbbgskqro0.cloudfront.net/MSGRMDS_1001/DATA/2015_114/EN1072174528M.IMG'
image_dir = 'data'
mess_img = os.path.join(image_dir, 'EN1072174528M.IMG')
downloader = urllib.request.URLopener()
if not os.path.isfile(mess_img):
    downloader.retrieve(imgurl, mess_img)

In [3]:
driver = MessengerMdisPds3NaifSpiceDriver(mess_img)

with driver as d:
    ale_position = d.sensor_position[0]
    ale_dict = ale.formatters.formatter.to_isd(d)

/usr/local/Caskroom/miniconda/base/envs/knoten/lib/python3.12/site-packages/osgeo/gdal.py:312: FutureWarning: Neither gdal.UseExceptions() nor gdal.DontUseExceptions() has been explicitly called. In GDAL 4.0, exceptions will be enabled by default.
  warnings.warn(


In [4]:
# Constructs a camera model using usgscsm
model="USGS_ASTRO_FRAME_SENSOR_MODEL"  # Make sure this matches your camera model
plugin = csmapi.Plugin.getList()[0]
isd = csmapi.Isd(mess_img)
warns = csmapi.WarningList()
if plugin.canModelBeConstructedFromISD(isd, model, warns):
    print("CONSTRUCTED CAMERA")
    camera = plugin.constructModelFromISD(isd, model)
else:
    print("CAN'T CONSTRUCT CAMERA")
    for item in warns:
        print(item.getMessage())

CONSTRUCTED CAMERA


In [5]:
# Ingest image and spiceinit it
cube = os.path.splitext(mess_img)[0] + '.cub'
isis.mdis2isis(mess_img, to=cube)
isis.spiceinit(cube, shape='ellipsoid')

CompletedProcess(args=['spiceinit', 'from=data/EN1072174528M.cub', 'shape=ellipsoid'], returncode=0, stdout='Group = Kernels\n  NaifIkCode                = -236820\n  LeapSecond                = $base/kernels/lsk/naif0012.tls\n  TargetAttitudeShape       = $messenger/kernels/pck/pck00010_msgr_v23.tpc\n  TargetPosition            = $messenger/kernels/tspk/de423s.bsp\n  InstrumentPointing        = ($messenger/kernels/ck/msgr_1504_v01.bc,\n                               $messenger/kernels/ck/msgr_mdis_sc040812_150430-\n                               v1.bc,\n                               $messenger/kernels/ck/msgr_mdis_gm040819_150430-\n                               v1.bc, $messenger/kernels/fk/msgr_v231.tf)\n  Instrument                = $messenger/kernels/ik/msgr_mdis_v160.ti\n  SpacecraftClock           = $messenger/kernels/sclk/messenger_2548.tsc\n  InstrumentPosition        = $messenger/kernels/spk/msgr_20040803_20150430_o-\n                              d431sc_2.bsp\n  InstrumentAd

In [6]:
# Grab campt output on spiceinit'd cube

mess_lbl_file = os.path.splitext(mess_img)[0] + '.lbl'

if os.path.isfile(mess_lbl_file): # Delete if exists already
    os.remove(mess_lbl_file)

output = isis.campt(cube, to=mess_lbl_file)

with open(mess_lbl_file) as file:
    mess_pvl_str = file.read()

In [7]:
# and load it as a pvl
pvl_output = pvl.loads(mess_pvl_str)

In [8]:
# Grab body fixed coordinates from campt pvl output
campt_bodyfixed = pvl_output['GroundPoint']['BodyFixedCoordinate']
campt_bodyfixed = np.asarray(campt_bodyfixed.value) * 1000

# Grab body fixed coordinates from csm
ale_bodyfixed = csm.generate_ground_point(0, (256 - .5, 256 - .5), camera)
ale_bodyfixed = np.array([ale_bodyfixed.x, ale_bodyfixed.y, ale_bodyfixed.z])

# Compare the two body fixed coordinates
ale_bodyfixed - campt_bodyfixed

array([-166.86240809,   31.73164436,  -31.46452873])

In [9]:
# Grab spacecraft position from campt pvl output
campt_position = pvl_output['GroundPoint']['SpacecraftPosition']
campt_position = np.asarray(campt_position.value) * 1000 # km to meters

# Compare the two positions
ale_position - campt_position

array([[-160.95373014, -404.73483791,  456.13296515]])

In [11]:
# Grab InstrumentPosition table from the isis cube using tabledump
instrument_pos_table = str(isis.tabledump(from_=cube, name='InstrumentPosition').stdout)
parsed_string = instrument_pos_table.split(',')

In [12]:
# Grab sensor position from the table dump output
isis_j2000_pos = np.asarray([float(parsed_string[6][3:]), float(parsed_string[7]), float(parsed_string[8])])

# Grab ALE's sensor position
# ale_j2000_pos = np.asarray(j2000RotationPos)
ale_j2000_pos = ale_dict['instrument_position']['positions']

# Compare the two sensor positions that are in the j2000 reference frame
ale_j2000_pos - isis_j2000_pos

array([[ 0.47149248, -0.2471164 ,  0.33823364]])

In [13]:
# Grab velocities from the table dump output
isis_j2000_vel = np.asarray([float(parsed_string[9]), float(parsed_string[10]), float(parsed_string[11])])

# Grab ALE's velocities
ale_velocities = ale_dict['instrument_position']['velocities']
ale_j2000_vel = np.asarray(ale_velocities)

# Compare the two velocity lists that are in the j2000 reference frame
ale_j2000_vel - isis_j2000_vel

array([[-0.00138282,  0.00029637,  0.00021793]])

In [14]:
# Grab spacecraft position and body fixed look vector from csm
locus = camera.imageToRemoteImagingLocus(csmapi.ImageCoord(256 - .5, 256 - .5))
csm_bodyfixedLV = np.asarray([locus.direction.x, locus.direction.y, locus.direction.z])
csm_position = np.asarray([locus.point.x, locus.point.y, locus.point.z])

# Grab spacecraft position and body fixed look vector from campt pvl output
campt_bodyfixedLV = np.asarray(pvl_output['GroundPoint']['LookDirectionBodyFixed'])
campt_position = pvl_output['GroundPoint']['SpacecraftPosition']
campt_position = np.asarray(campt_position.value) * 1000 # km to m

# Compute the differences
print(csm_bodyfixedLV - campt_bodyfixedLV)
print(csm_position - campt_position)

[9.62932188e-06 8.28987816e-06 7.29129240e-06]
[-160.95373014 -404.73483791  456.13296515]
